<a href="https://colab.research.google.com/github/shintafiaa/Kampus-Merdeka-Pengembangan-ML/blob/main/Kampus_Merdeka_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#mengubah dataset menjadi dataframe
#membuang kolom 'judul_film'
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [12]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [13]:
df.tail()

,ringkasan_sinopsis,genre
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis
1004,"Atika (Meriam Bellina) mantan penyanyi tenar, ...",Romantis


In [14]:
#one-hot-encoding dan membuat dataframe baru
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [9]:
#mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [10]:
#split data training dan testing
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [15]:
#Tokenisasi
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
#mengonversi setiap sampel menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [16]:
#Arsitektur model dengan layer embedding
#dimensi embedding 16, dimensi input sebesar num_words
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
#training
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 27s - loss: 1.6100 - accuracy: 0.1891 - val_loss: 1.6096 - val_accuracy: 0.1990
Epoch 2/30
26/26 - 25s - loss: 1.6073 - accuracy: 0.2289 - val_loss: 1.6082 - val_accuracy: 0.2189
Epoch 3/30
26/26 - 25s - loss: 1.5811 - accuracy: 0.3396 - val_loss: 1.5808 - val_accuracy: 0.2239
Epoch 4/30
26/26 - 25s - loss: 1.4900 - accuracy: 0.4726 - val_loss: 1.5760 - val_accuracy: 0.2687
Epoch 5/30
26/26 - 25s - loss: 1.0486 - accuracy: 0.6418 - val_loss: 1.5776 - val_accuracy: 0.3134
Epoch 6/30
26/26 - 25s - loss: 0.6762 - accuracy: 0.7861 - val_loss: 2.0223 - val_accuracy: 0.3284
Epoch 7/30
26/26 - 25s - loss: 0.2937 - accuracy: 0.9167 - val_loss: 3.0051 - val_accuracy: 0.3333
Epoch 8/30
26/26 - 25s - loss: 0.1692 - accuracy: 0.9515 - val_loss: 3.2453 - val_accuracy: 0.3035
Epoch 9/30
26/26 - 25s - loss: 0.1171 - accuracy: 0.9689 - val_loss: 3.2390 - val_accuracy: 0.2985
Epoch 10/30
26/26 - 25s - loss: 0.0502 - accuracy: 0.9925 - val_loss: 3.7609 - val_accuracy: 0.3234
Epoch 11/

Terjadi overfitting. Sudah cukup bagus karena data yang dimiliki berjumlah 1000. 